<a href="https://colab.research.google.com/github/Ofir-Manor/Long3/blob/main/316084623-311288203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"> HW3 Major</h1>
<h1 align="center"> Regression</h1>

# Setting the Environment

In [ ]:
#@title Setting the Environment:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pylab
from typing import Union
params = {
    'xtick.labelsize': 18,
    'ytick.labelsize': 18,
    'axes.titlesize' : 22,
    'axes.labelsize' : 20,
    'legend.fontsize': 18,
    'legend.title_fontsize': 22,
    'figure.titlesize': 24
 }
pylab.rcParams.update(params)

## Preliminary: Updated Data Loading

In [ ]:
#@title Loading the dataset and the code files:
# load the csv file:
!rm *.csv
from google.colab import files
print("Please load the raw csv data: HW3_data")
uploadedData = files.upload()
filename = list(uploadedData)[0]

In [ ]:
#@title Create DF and split it:

dataset = pd.DataFrame(pd.read_csv(filename))
from sklearn.model_selection import train_test_split
random_state = 3 + 23
train_raw, test_raw = train_test_split(dataset, test_size=0.2, random_state=random_state)

In [ ]:
#@title Use `prepare` function: